Please remember to update the token. For multiplayer to work, each player needs a different token. Please feel free to use multiple tokens in the same group.

In [ ]:
#Update your token
STUDENT_TOKEN = 'GROUP 1'

# Base Code

There should be no need to change the code below.

In [ ]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'
  
  def get_board(self):
    return json.loads(self.state)

  def get_winner(self):
    return None

  def actions(self):
    return []

  def print(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('Creating new game...')
    id = new_game(game_type, multi_player)

  print('Joining game with id: %s' % id)
  player = join_game(game_type, id)

  print('Playing as %s' % player)

  game = game_state(game_type, id, GameClass)
  print('Waiting for the other player to join...')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('Server says there is a draw')
      else:
        print('Server says your agent ' + ('WON' if game.player == player else 'LOST'))
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      print(move)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

In [ ]:
def game_loop_test(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    id = new_game(game_type, multi_player)

  player = join_game(game_type, id)


  game = game_state(game_type, id, GameClass)
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    if game.is_end():
      if game.player == '-':
        return 0
      else:
        if game.player == player:
          return 1
        else: 
          return 0
      return
    if game.player == player:
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
    else:
      time.sleep(2)

There is almost certainly no need to change this code, but if you must you can.

In [ ]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

class Clobber(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    return [] # this should return the possible actions

  def get_winner(self):
    return '.' # this should return the actual winner

  def other_player(self):
    if self.player == 'O': return 'X'
    if self.player == 'X': return 'O'

  def print_game(self):
    for row in self.get_board():
      print(row)


# Random Solver

Instead of changing this code for a random solver, write new code that also returns the next move. For example, create a new "amazing_solver".

Solvers need to return a list or tuple of 3 elements: row, col and dir. Dir can be 'R', 'L', 'U', 'D'

In [ ]:
def random_solver(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  opts = []
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          opts.append((row, col, 'D'))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          opts.append((row, col, 'R'))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          opts.append((row, col, 'U'))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          opts.append((row, col, 'L'))
  return opts[random.randint(0, len(opts) - 1)]

This code is the one that starts the game. In different games you will need to change the class (Clobber is the one for clobber game).

The 'multi_player' variable can be True or False. If you set multi_palyer to True, you will need to provide a game_id against which to play or if using None a new game will be created (which another player can join).

In [ ]:
game_loop(random_solver, Clobber, 'clobber', multi_player=False, id=None)

# Solver V12 (The Chosen One)

In [ ]:
def number_of_moves(input_board, player, other_player): # This function calculates the number of moves available in the board for the player
  board = deepcopy(input_board) # This creates a full copy of the board for us to manipulate
  moves = 0 # We initiate our moves variable
  for row in range(len(board)): # We iterare throught the board using for loops
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          moves += 1 # If we find a possible move, we add 1 to our moves variable
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          moves += 1
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          moves += 1
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          moves += 1
  return moves # We return our moves variable

def available_moves(input_board, player, other_player): # This function returns a list with all the available moves that the player can do
  board = deepcopy(input_board) # This creates a full copy of the board for us to manipulate
  a_moves = [] # We initiate our moves list
  for row in range(len(board)): # We iterate through the board using for loops
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          a_moves.append((row, col, 'D')) # If we find a possible move we append it to the list
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          a_moves.append((row, col, 'R'))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          a_moves.append((row, col, 'U'))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          a_moves.append((row, col, 'L'))
  return a_moves # We return the list of moves

def move_preview_v12(input_board, player, move): # This function returns a new board to simulate what would happend if we make the selected move
  board = deepcopy(input_board) # This creates a full copy of the board for us to manipulate
  board[move[0]][move[1]] = '.' # We replace the spot on the board with a period
  if move[2] == 'D': # Depending on how we are moving, we replace the other_player's spot with our player
    board[move[0] + 1][move[1]] = player
  if move[2] == 'R':
    board[move[0]][move[1] + 1] = player
  if move[2] == 'U':
    board[move[0] - 1][move[1]] = player
  if move[2] == 'L':
    board[move[0]][move[1] - 1] = player
  return board # We return the new board

def solver_v12(game): # This is the main function of the algorithm
  board = game.get_board() # We get the board, player and other_player from the game input
  player = game.player
  other_player = game.other_player()
  value, move = max_value_v12(board, 3, player, other_player) # We initiate the MiniMax
  return move # We return the selected move

def max_value_v12(board, depth, player, other_player): # This is our max function
  my_moves = number_of_moves(board, player, other_player) # We calculate our number of moves and the opponent's number of moves
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0: # We check to see if the depth is equal to 0
    if my_moves > other_moves: # If we have more moves than the opponent we return 1, else we return -1
      return 1, None
    else:
      return -1, None
  if my_moves == 0: # If our moves are 0, the game is over and we return 1. Else, the game is over and we return -1
    return -1, None
  if other_moves == 0:
    return 1, None
  win_options = {} # We create our win_options and lose_with_honor dictionary
  lose_with_honor = {}
  for action in available_moves(board, player, other_player): # We iterate through our list of available moves
    preview = move_preview_v12(board, player, action) # We get the preview board according to our selected move
    v2, a2 = min_value_v12(preview, depth-1, other_player, player) # We initiate the min function for that move
    if v2 == 1: # If the function returns victory, we add the move to the win_options dictionary according to our heuristic, which consist of leaving the opponent with the least amount of moves as possible.
      win_options[action] = number_of_moves(preview, other_player, player)
    elif v2 == -1: # Else, we add it to our lose_with_honor dictionary
      lose_with_honor[action] = number_of_moves(preview, other_player, player)
  if len(win_options) == 0: # If we don't have win options, we get the min from our lose_with_honor
    move = min(lose_with_honor, key=lose_with_honor.get)
  else: # Else, we use our win_options
    move = min(win_options, key=win_options.get)
  value = v2 # We define our value
  return value, move # We return our move and value
  
def min_value_v12(board, depth, player, other_player): # This is our min function
  my_moves = number_of_moves(board, player, other_player) # We calculate our number of moves and the opponent's number of moves
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0: # We check to see if the depth is equal to 0
    if my_moves > other_moves: # If the opponent has more moves than us we return -1, else we return 1
      return -1, None
    else:
      return 1, None
  if my_moves == 0: # If the opponent's moves are 0, the game is over and we return -1. Else, the game is over and we return 1
    return 1, None
  if other_moves == 0:
    return -1, None
  win_options = {} # We create our win_options and lose_with_honor dictionary
  lose_with_honor = {}
  for action in available_moves(board, player, other_player): # We iterate through our list of available moves
    preview = move_preview_v12(board, player, action) # We get the preview board according to our selected move
    v2, a2 = max_value_v12(preview, depth-1, other_player, player) # We initiate the max function for that move
    if v2 == -1: # If the function returns victory for the other_player, we add the move to the win_options dictionary according to our heuristic, which consist of leaving the opponent with the least amount of moves as possible.
      win_options[action] = number_of_moves(preview, other_player, player)
    elif v2 == 1: # Else, we add it to our lose_with_honor dictionary
      lose_with_honor[action] = number_of_moves(preview, other_player, player)
  if len(win_options) == 0: # If we don't have win options, we get the min from our lose_with_honor
    move = min(lose_with_honor, key=lose_with_honor.get)
  else: # Else, we use our win_options
    move = min(win_options, key=win_options.get)
  value = v2 # We define our value
  return value, move # We return our move and value

# Tester

In [ ]:
game_loop(solver_v12, Clobber, 'clobber', multi_player=False, id=None)

In [ ]:
i = 0
won = 0
while i < 5:
  won += game_loop_test(solver_v12, Clobber, 'clobber', multi_player=False, id=None)
  i += 1
  print(i)
print("The algorithm won ", won, " times!")